# Compute Mobile Odds Ratio

Assess treatment effect on mobile proportion using mobile:immobile odds ratio (e.g. Ly et al. Figure 5) 

In [1]:
#identify treatments and control labels:
myTreatments<-c('mbcd')
myControl<-'untreated'

In [2]:
#load class/treatment data frame
df<-readRDS('../generate_ImmobileMobile/mbcd-untreated-exampleDF.rds')

As a first example, we compute odds ratio for the mbcd/untreated single data sets. The R implementation of Fisher's exact test provides the odds ratio estimate and corresonding CI.

In [3]:
myFish<-list()
for(i in seq_along(myTreatments)) {
    #prepare two-dimensional contingency table for each treatment
    mytreats<-c(myControl,myTreatments[i])
    dfSel<-df[df$treatment%in%mytreats,]
    dfSel$treatment<-factor(dfSel$treatment,levels = mytreats)
    dfSel$class<-factor(dfSel$class,levels=c('immobile','mobile'))
    mytable<-table(dfSel$class,dfSel$treatment)
    
    #run Fisher's exact test
    ft<-fisher.test(mytable)
    print(myTreatments[i])
    print(ft)
    
    #gather results in a data frame including the odds ratio estimate and the min/max values of the CI
    myFish[[i]]<-data.frame(odr=ft$estimate,clo=ft$conf.int[1],chi=ft$conf.int[2],
                            treatment=myTreatments[i])
    }

[1] "mbcd"

	Fisher's Exact Test for Count Data

data:  mytable
p-value = 5.896e-06
alternative hypothesis: true odds ratio is not equal to 1
95 percent confidence interval:
 1.159443 1.460315
sample estimates:
odds ratio 
  1.301145 



In [4]:
#results from different treatments can be collected in a data frame
dfOR<-do.call(rbind,myFish)
dfOR

,odr,clo,chi,treatment
,<dbl>,<dbl>,<dbl>,<chr>
odds ratio,1.301145,1.159443,1.460315,mbcd


In Ly et al. Figure 5, the mobile/immobile proportions for a given treatment were gathered from several experimental sessions. We stratified the mobile/immobile counts by experimental session and used the R implementation of the Cochran-Mantel-Haenszel test to generate a common odds ratio and corresponding CI.

Here, for illustration purposes, we add a column to the treatment/class data frame identifying the "experiment number", then duplicate the mbcd/mobile counts and assign a different "experiment number" to the copy.

In [5]:
df$expnum<-"260"

dfbis<-df

dfbis$expnum<-'99'

dfmock<-rbind(df,dfbis)

In [6]:
table(dfmock$treatment,dfmock$class,dfmock$expnum)

, ,  = 260

           
            immobile mobile
  mbcd           819   2336
  untreated      849   1861

, ,  = 99

           
            immobile mobile
  mbcd           819   2336
  untreated      849   1861


In [7]:
#In general, each treatment will have a different set of "experiment numbers", 
#we use this list to extract the correct ones in each case
myExpList<-list(mbcd=c('260','99'))

In [8]:
myMant<-list()
for(i in seq_along(myTreatments)) {
    #prepare stratified contingency table
    mytreats<-c(myControl,myTreatments[i])
    mydf<-dfmock[dfmock$treatment%in%mytreats
                & dfmock$expnum%in%myExpList[[myTreatments[i]]],]
    mydf$treatment<-factor(mydf$treatment,levels = mytreats)
    mytable<-(table(mydf$treatment,mydf$class,mydf$expnum)*1.0)

    #run Mantel-Haenszel test
    mtb<-mantelhaen.test(mytable)
    print(myTreatments[i])
    print(mtb)

    #gather results in a data frame including the odds ratio estimate and the min/max values of the CI
    myMant[[i]]<-data.frame(odr=mtb$estimate,clo=mtb$conf.int[1],chi=mtb$conf.int[2],
                            treatment=myTreatments[i])
    }
#collect results from different treatments into a single data frame
dfOR<-do.call(rbind,myMant)

[1] "mbcd"

	Mantel-Haenszel chi-squared test with continuity correction

data:  mytable
Mantel-Haenszel X-squared = 41.036, df = 1, p-value = 1.494e-10
alternative hypothesis: true common odds ratio is not equal to 1
95 percent confidence interval:
 1.200713 1.410137
sample estimates:
common odds ratio 
         1.301219 



In [9]:
dfOR

,odr,clo,chi,treatment
,<dbl>,<dbl>,<dbl>,<chr>
common odds ratio,1.301219,1.200713,1.410137,mbcd
